In [2]:
'''This script is for downloading sentinel data using API based on the user inputs'''

'This script is for downloading sentinel data using API'

In [3]:
#### import libraries
import os
import json
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt, make_path_filter, placename_to_wkt
from datetime import date

In [4]:
#### base path
base_path = os.getcwd()

#### user inputs
# for accessing sentinel API
username = 'abnajmi2'
password = 'Copernicus@04'
# study area
city_name = 'Enschede' # define if study area have to be selected by city name
geojson_path = os.path.join(base_path,'filename.geojson') # define file path if study area have to be selected using polygon
point = 'POINT (41.9 12.5)' # define if study area have to be selected by using point (in WKT)

# date
single_date = ('20151219', date(2015, 12, 29)) # if single date is defined
range_date = ('20170801','20170830') # if duration is defined then user have to mention starting and ending date

# output path to save the downloaded data
output_path = os.path.join(base_path,'result')
txt_path = os.path.join(output_path,'metadata.txt')

In [5]:
#### accessing api through uerid and password
api = SentinelAPI(username, password, 'https://apihub.copernicus.eu/apihub')

#### search by footprint (study area), time, and SciHub query keywords
# study area
# footprint = geojson_to_wkt(read_geojson(geojson_path))
# footprint = placename_to_wkt(city_name)[0]
footprint = point

#### querying available products based on user defined criteria
products = api.query(footprint,
                     date=single_date,
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 30))

In [6]:
#### converting products output into geodataframe
product_gpd = api.to_geodataframe(products)
product_id = product_gpd['uuid'][0] # here i have hard coded the uuid from geodataframe for this implementation

#### Get the product's full metadata available on the server
metadata = api.get_product_odata(product_id, full=True)

In [6]:
#### download all products
'''
It will download all available products either online or offline. It will download online products, while concurrently
triggering the retrieval of offline products from the LTA in the background. download_all() terminates when all
products have been retrieved from the LTA and downloaded.
'''
# api.download_all(products, directory_path=output_path)

#### download slected product
'''
Copernicus Open Access Hub no longer stores all products online for immediate retrieval. Offline products can be
requested from the Long Term Archive (LTA) and should become available within 24 hours. Copernicus Open Access
Hub's quota currently permits users to request an offline product every 30 minutes.
'''
is_online = api.is_online(product_id) # checking the data whether it is online or offline
if is_online:
    print(f'Product {product_id} is online. Starting download.')
    api.download(product_id, directory_path=output_path)
else:
    print(f'Product {product_id} is not online.') 
    api.trigger_offline_retrieval(product_id) # The retrieval of offline products from the LTA is triggered

Product 835405b7-2c17-4681-b743-fb9d7bb5591a is not online.


In [11]:
# creating .txt file for metadata
with open(txt_path, 'w') as f:
    for key, value in metadata.items(): 
        f.write('%s:%s\n' % (key, value))